In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0022041797637939453
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 9.751728251181042
200000 16.945405127430835
300000 19.11426766181313
400000 19.391629464444815
500000 19.26833844392219
600000 19.865605039396236
700000 18.78760129599203
800000 18.710658549572933
900000 18.93150576644752
1000000 16.652780379783774
1100000 8.87769671200006
1200000 18.53486770257315
1300000 18.471490949477786
1400000 18.531100382590786
1500000 18.71374692900746
1600000 19.72539702374608
1700000 18.535808197051725
1800000 18.87226554978682
1900000 19.024827268056242
2000000 19.189666710036857
2100000 5.068423164639003
2200000 21.769692587331487
2300000 23.527343513718794
2400000 23.015972370217728
2500000 23.123179474969373
2600000 23.554116805911086
2700000 22.470170646652956
2800000 21.956795721435295
2900000 22.328200444058243
3000000 22

29400000 16.96204275572558
29500000 17.367963369893214
29600000 16.212785705731754
29700000 11.50363419227254
29800000 11.593061462583382
29900000 16.147174623378277
30000000 16.760959315474164
30100000 16.77946168936618
30200000 18.01311383602136
30300000 17.62227359664395
30400000 18.306145141557185
30500000 12.033291910018665
30600000 14.200527315801477
30700000 18.04879666413831
30800000 18.36470696782066
30900000 18.21591694653383
31000000 9.663240483665987
31100000 16.196892258770696
31200000 18.42828812347072
31300000 18.25778180210933
31400000 18.074859197050667
31500000 17.272660602833085
31600000 16.694984295066618
31700000 16.86090503225308
31800000 5.441093651149348
31900000 17.06897582781258
32000000 16.422429385545094
32100000 16.852510038978355
32200000 17.229099155939174
32300000 17.962273304617995
32400000 17.992616397070137
32500000 15.708079454450141
32600000 10.310802811256078
32700000 17.538429823739587
32800000 17.827281128429718
32900000 18.253790331997298
330000

59100000 15.201396705448287
59200000 15.781491445774689
59300000 15.513184446029719
59400000 15.820373275748622
59500000 16.358409168098667
59600000 15.893053741900392
59700000 15.832303506415474
59800000 15.559163969718783
59900000 8.669022831012684
60000000 12.92217406338954
60100000 14.23860365868588
60200000 14.567193197635813
60300000 13.930474822039875
60400000 13.098540791227714
60500000 6.7965639818132715
60600000 14.334206889253597
60700000 14.10040027087689
60800000 13.6306424886765
60900000 13.884306593191873
61000000 8.385351623739085
61100000 14.00736804030862
61200000 15.57794922380374
61300000 15.156089643581643
61400000 15.978281055948827
61500000 16.163227742201972
61600000 15.722019770006481
61700000 15.179169044448942
61800000 15.563153317980234
61900000 15.283167623831204
62000000 6.310439898104419
62100000 13.750555261366873
62200000 13.600377455948994
62300000 13.550250362831559
62400000 13.570970281314633
62500000 6.719716474080995
62600000 12.788780023042207
627

88700000 11.813061597400177
88800000 7.0850912610962045
88900000 10.2528538070469
89000000 12.431348215280229
89100000 8.619764396295123
89200000 7.982479774930581
89300000 11.842835478882678
89400000 11.191687340737928
89500000 7.049432482593985
89600000 11.849758824678831
89700000 11.85507719588591
89800000 12.252119178040342
89900000 12.902095355288095
90000000 13.268698943523152
90100000 13.014922676068084
90200000 13.19307744422384
90300000 14.099414480958167
90400000 12.74467541544202
90500000 12.814054256545974
90600000 12.050418182399099
90700000 11.665628654491577
90800000 11.867183567040849
90900000 4.944392145528318
91000000 11.708460712832888
91100000 11.850597250638065
91200000 3.9654069678819273
91300000 11.859795963688992
91400000 11.61776427017991
91500000 5.5991319564334345
91600000 12.172482212974122
91700000 11.581807260515292
91800000 11.500643649625154
91900000 12.246875966844911
92000000 12.49833320542791
92100000 12.879275691552158
92200000 12.864583025867105
923

117800000 3.270136589408056
117900000 4.118283627275827
118000000 8.932305930621473
118100000 9.825630105787297
118200000 9.804811179928748
118300000 9.780540032736411
118400000 9.931067568679131
118500000 9.915824717238332
118600000 10.111302253904892
118700000 10.18142028749277
118800000 10.818043210618296
118900000 10.927102633424667
119000000 9.675419603128578
119100000 9.750413823085076
119200000 9.344366838610636
119300000 9.403153365338367
119400000 9.472576933781188
119500000 9.620294361251288
119600000 9.108454708259682
119700000 9.484419227010711
119800000 3.147921942396018
119900000 1.756779532269281
120000000 6.8822657016308755
120100000 9.172962530871551
120200000 9.12057882160222
120300000 9.528447720081047
120400000 9.378459669059934
120500000 9.468107976797983
120600000 9.257049540017245
120700000 9.874629721978312
120800000 9.719356945166426
120900000 11.486055358458994
121000000 11.015751832913905
121100000 10.482428338934325
121200000 10.349348522619064
121300000 10.

146700000 6.588412467130515
146800000 7.804089073401521
146900000 8.048279987887302
147000000 8.138197938022346
147100000 8.161914970415225
147200000 8.284268852915096
147300000 8.08819305795586
147400000 8.846302036557868
147500000 8.734240547114043
147600000 8.871048627540794
147700000 8.322158826367756
147800000 7.8668731241968235
147900000 7.926075699262684
148000000 7.994295397587841
148100000 7.901429981756449
148200000 7.7819445612262035
148300000 7.426189837309285
148400000 3.8158577341905056
148500000 6.277423964862008
148600000 5.219674506913374
148700000 3.441902320784509
148800000 7.665545283880966
148900000 7.714027721916603
149000000 7.88876391534121
149100000 7.9331777121771125
149200000 7.911545257376954
149300000 7.830724447548828
149400000 8.22666777615239
149500000 8.691790727444904
149600000 8.625018189934714
149700000 8.547100434951211
149800000 7.88000258352873
149900000 7.9096626919971404
150000000 7.909803284936002
150100000 7.8836153237139825
150200000 7.807113

176000000 6.710461497014243
176100000 6.672504844070653
176200000 6.609293784713808
176300000 6.4332056289764346
176400000 6.4762778930053635
176500000 6.284908574704367
176600000 6.126022521500832
176700000 5.8521966541261055
176800000 4.796112142320843
176900000 3.8542181904248
177000000 5.868688346737212
177100000 5.917930893551798
177200000 3.197230054709957
177300000 4.865985208264944
177400000 5.815117675311127
177500000 5.886079658579814
177600000 2.5100629683957005
177700000 5.933504907433381
177800000 5.851720143470341
177900000 6.410104210162999
178000000 6.2425101825217135
178100000 6.516943640023293
178200000 6.48311821119088
178300000 6.415007144771338
178400000 6.275430370911582
178500000 6.274767649372979
178600000 6.4864264835651
178700000 5.904862011393475
178800000 5.873686563640919
178900000 2.7447300688304916
179000000 5.859015162639452
179100000 5.920045009651398
179200000 5.668247383920183
179300000 2.409980112103357
179400000 5.774885960269782
179500000 5.9806830

205200000 4.446738404128048
205300000 2.098822674958837
205400000 4.494578762029171
205500000 4.071951162692411
205600000 4.250701365295328
205700000 3.9079055953102646
205800000 4.19620426840659
205900000 1.4424697123498367
206000000 4.043063924444088
206100000 4.216597216211087
206200000 3.8661307546230756
206300000 4.167570845690346
206400000 4.3194439471161274
206500000 1.76713118405957
206600000 4.697640713166136
206700000 4.431108937118839
206800000 4.717880253985802
206900000 4.328861136424478
207000000 4.740020346432806
207100000 4.341422334045886
207200000 4.719936571696647
207300000 2.928500666876524
207400000 3.1568601437006216
207500000 4.411859833205897
207600000 3.9849257574897625
207700000 4.071277307732333
207800000 3.7861986472145497
207900000 3.0459927334994545
208000000 2.649654888999203
208100000 3.829153243715976
208200000 4.116071562699427
208300000 4.012166710364967
208400000 4.312331973733455
208500000 3.817824004965741
208600000 2.224615922339052
208700000 4.59

234100000 2.2890606329282988
234200000 2.160177648319876
234300000 2.114212744547985
234400000 2.108934951323933
234500000 2.0202156744645356
234600000 0.8731727850253794
234700000 1.9730344300173057
234800000 2.0410859126250407
234900000 2.2208489240487324
235000000 2.082733443141186
235100000 2.245994485301167
235200000 2.156315453373792
235300000 2.2924563278554158
235400000 0.9053259544482173
235500000 2.309898564413592
235600000 2.1479391074243064
235700000 2.239281062869487
235800000 0.9151469107771735
235900000 2.069864941750218
236000000 2.180048497488525
236100000 1.9861976948626805
236200000 2.105498026203261
236300000 2.019853771896422
236400000 2.101767369188192
236500000 1.8608620477605706
236600000 1.0645500263609886
236700000 1.6727259853298029
236800000 1.8499253287077115
236900000 2.076759534207932
237000000 2.0159562546660523
237100000 2.0941228938518606
237200000 1.9826463475305973
237300000 2.116094578605881
237400000 1.8356208072567448
237500000 1.015449352430584
2

262100000 0.0007124125805795193
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 4.8998000309864684e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
